# Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams

This notebook implements the methods described in [Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4040303), presented at the [Computational Legal Studies Conference](https://cclaw.smu.edu.sg/events/computational-legal-studies-2022), March 2022, hosted by the Center for Computational Law at Singapore Management University. Here is a link to the presentation's [slide deck](https://docs.google.com/presentation/d/15fxG3zoZSdUfmxFuVQZd2_MrDe1Z1rcsXwBDIvFIy3U/edit?usp=sharing), including image credits. 

Some of the data presented here differ slightly from that found in the version presented at the CLS Conference and in the slide deck above. This is due to additional work done in response to feedback received after presenting the paper. They now show the difference in performance between pseudo-random and machine ordering _after_ both machine and human markings are converted into z-scores. The translation of both scores into z-scores allowed the machine score to be compared with the human score using intraclass correlation ICC and Cohen's kappa. The CLS presentation compared the machine scores only to pseudo-random scores without translating the human scoring and after the machine score was transformed into a numerical score based on a standard grading scale (e.g., 90, 80, etc.). Older versions of this notebook with prior results can be found [here](https://github.com/colarusso/free-response-scoring/commits/main/Score%20Exams.ipynb).

**Paper Summary**

> This paper presents a novel method for unsupervised machine scoring of short answer and essay question responses, relying solely on a sufficiently large set of responses to a common prompt, absent the need for pre-labeled sample answers—given said prompt is of a particular character. That is, for questions where “good” answers look similar, “wrong” answers are likely to be “wrong” in different ways. Consequently, when a collection of text embeddings for responses to a common prompt are placed in an appropriate feature space, the centroid of their placements can stand in for a model answer, providing a lodestar against which to measure individual responses. This paper examines the efficacy of this method and discusses potential applications.
>
>Current methods for the automated scoring of short answer and essay questions are poorly suited to spontaneous and idiosyncratic assessments. That is, the time saved in grading must be balanced against the time required for the training of a model. This includes tasks such as the creation of pre-labeled sample answers. This limits the utility of machine grading for single classes working with novel assessments. The method described here eliminates the need for the preparation of pre-labeled sample answers. It is the author’s hope that such a method may be leveraged to reduce the time needed to grade free response questions, promoting the increased adoption of formative assessment esp. in contexts like law school instruction which traditionally have relied almost exclusively on summative assessments.
>
>Ranking by the algorithm is found to be statistically significant when compared to a pseudo-random shuffle. To determine how similar a list’s order was to that produced by a human grader, the lowest number of neighbor swaps needed to transform the ordering of these lists into that of the human ordering was calculated. For a dataset including more than one thousand student answers to a set of thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors, the p-value for a paired t-test of the two populations’ swaps, with the pseudo-random group acting as the untreated group and the machine-grader acting as the treatment, came to 0.000000334, allowing us to reject the null hypothesis that the machine's ordering is equivalent to a random shuffle. Additionally, the Cohen’s d for the number of swaps between the pseudo-random ordering and machine ordering was found to be large (i.e., 1.03).

This notebook was used to obtain the results referenced above. Student exam answers and their associated grade data were acquired and this research conducted after approval was granted by Suffolk University’s Office of Research and Sponsored Programs (ORSP) which oversees all human subject research at Suffolk University.

This notebook implements the primary steps of the method described in the paper, namely:  

1. Produce an embedding for each answer that captures as much of the relevant information as possible.
2. Find the centroid for all of the embeddings in your population of answers and impute the location of a  “correct” model answer.
3. Measure the distance between each answer’s embedding and the “correct” answer (e.g., the answers’ centroid or medoid).
4. Convert the answers’ distances from the model answer into z-scores for the population of answers.
5. Translate these z-scores into some known grading scale.
6. Order the answers according to this scale.
7. Compare these orderings to random orderings by seeing how many times you have to change their rankings to obtain the same ordering as one would get if they were ordered by their human-assigned grades.

_Note: The methods described here are the subject of USPTO Patent Application Serial No. 17/246,563._

## Contents

_These, and other internal links (e.g., "back to contents"), may not work if you are viewing the preview of this notebook on GitHub.com. Links to other notebooks, external sites, and exam questions should, however, be okay._

This notebook has three sections:

- [Data: Student Exam Answers](#Data:-Student-Exam-Answers)
- [Code:Code: Libraries and Novel Functions](#Code:-Libraries-and-Novel-Functions)
- [Results](#Results)


## Data: Student Exam Answers

More than one thousand exam answers were obtained as PDF files containing answers to thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors. Each PDF corresponded to a single student exam. These PDFs were parsed to extract their answers, their contents converted into XML of the following format.

```
<EXAM>
    <STUDENT id='ID'>00000000</STUDENT>
    <QUESTION id='Q1'>
        <![CDATA[
            text of written answer to question one
        ]]>
    </QUESTION>
    <QUESTION id='Q2'>
        <![CDATA[
            text of written answer to question two
        ]]>
    </QUESTION>
    <QUESTION id='Q3'>
        <![CDATA[
            text of written answer to question three
        ]]>
    </QUESTION>
</EXAM>
```

These translated files were reviewed by hand and reformatted as needed to correct any formatting errors. Each XML file was then read into a csv file for its associated exam as a single row with their columns corresponding to each of its elements (e.g., ID, Q1, Q2). Additionally, a column stating the number of words contained in each question was appended to the csv file (e.g., size_Q1, size_Q2). E.g., 

|ID|Q1|Q2|size_Q1|size_Q2|
|--|--|--|-------|-------|
|00001|text of 1's ans to q1|text of 1's ans to q2|6|6|
|00002|text of 2's ans to q1|text of 2's ans to q2|6|6|

Instructors also provided scores for each exam question. These were placed in a csv for each exam with the scores on each question associated to the exam ID. E.g., 

|ID|Q1|Q2|
|--|--|--|
|00001|96|93|
|00002|83|89|

A PDF did not extist for every exam as some exams were hand-written. Also, for the _Property Instructor A_ exam, the IDs of two PDFs did not macth any of the IDs provided in the score sheet. To address such issues, you will see that we merge the two data tables above using the default behavior of Panda's merge method (i.e., we kept only the intersection of ID, those found in both files). Consequently, we did not score the exams for which no PDF existed or those for which we could not find a score on the intructor-provided list of scores.

To see the code used to assist in this processing see the following notebook: [Prep Exams](Prep%20Exams.ipynb).

In keeping with the wishes of those instructors who provided exams, only three of the exams are available here for review (i.e., [property_instructor_A](https://colarusso.github.io/free-response-scoring/data/property_instructor_A/property_instructor_A.docx), [property_instructor_B](https://colarusso.github.io/free-response-scoring/data/property_instructor_B/property_instructor_B.docx), and [crim_instructor_E](https://colarusso.github.io/free-response-scoring/data/crim_instructor_E/crim_instructor_E.docx)). Another two are on file with the author and may be shared upon request and the assent of their authors. In keeping with the author's wishes, the remaining exam will not be shared. Subject to constraints imposed by the Family Educational Rights and Privacy Act (FERPA), at least three of the answer sets may be shared upon request. One of these is linked to an exam requiring instructor assent to be shared. I am in the process of reviewing and cleaning these data for PII. You can see those data that have been cleaned and shared without need for instructor assent [here](https://github.com/colarusso/free-response-scoring/tree/main/data) and track my progress [here](https://twitter.com/Colarusso/status/1494781654237425666?s=20&t=g0Zt5f3wDnVys6oH3nVSGQ).

Please note that the exam questiones shared at the links above are docx files and have been redacted to exclude the instructor's name and text that does not include the scored question prompts (e.g., multiple choice questions). 

To facilitate retrieval of the exam answers which were stored in various folders, a list of dictionaries is defined. Each dictionary defines the folder name where csv files can be found as well as the names of its relevant columns for later consideration (i.e., their ID and those questions to be scored). To avoid sharing of this data publicly, as seen below, these files are not included in this repository and were located outside of this repository's folder (i.e., `../data/`). 

In [1]:
exams = [
            {"folder":"../data/property_instructor_A","columns":["ID","SHORT_ANS","Q1","Q2"]},
            {"folder":"../data/property_instructor_B","columns":["ID","Q1","Q2"]},
            {"folder":"../data/environ_instructor_B","columns":["ID","Q2"]},
            {"folder":"../data/PR_instructor_C","columns":["ID","Q1","Q2"]},
            {"folder":"../data/contracts_instructor_D","columns":["ID","Q1","Q2","Q3"]},
            {"folder":"../data/crim_instructor_E","columns":["ID","Q1","Q2"]}
          ]

[back to contents](#Contents)

## Code: Libraries and Novel Functions

### Python Libraries et al.

First we'll load the following libraries. These are needed to perform our scoring. 

In [2]:
import csv
import pandas as pd
import numpy as np
from numpy import var
from math import sqrt
import spacy
from scipy import stats
from sklearn.preprocessing import normalize
from sklearn.metrics import cohen_kappa_score
import pingouin as pg

# https://spacy.io/models/en#en_core_web_lg
nlp1 = spacy.load('en_core_web_lg') 
# https://github.com/explosion/spacy-transformers/tree/88814f5f4be7f0d4c784d8500c558d9ba06b9a56
nlp2 = spacy.load("en_trf_distilbertbaseuncased_lg") 
nlp3 = spacy.load("en_trf_robertabase_lg") 

In [3]:
def score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1):

    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    M = []  
    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    R = []

    # for every exam in the list of exams  
    for exam in exams:

        print("\n=========================================================")
        print(exam["folder"])
        print("=========================================================\n")

        # load this exam's answer texts
        texts = pd.read_csv("%s/texts_redacted.csv"%exam["folder"], encoding="utf-8").sample(frac=1)
        # load this exam's grades
        actual = pd.read_csv("%s/actual_redacted.csv"%exam["folder"], encoding="utf-8").sample(frac=1)
        
        # print the relevant columns for this exam (i.e., ID and questions to score)
        print(exam["columns"])        

        # score the questions listed above
        output_df = grade(texts[texts["ID"].isin(actual["ID"])][exam["columns"]],model,normv,score,goal)
        
        if score==2:
            # convert human grades to z scores if score==2
            print("Computing z scores for human scoring")
            actual = z_scores_from_grades(actual)
            # create a df of the format needed for ICC
            df_output_df = output_df.copy()
            df_output_df["judge"] = "Machine"
            df_actual = actual.copy()
            df_actual["judge"] = "Human"
            df_icc = pd.concat([df_actual,df_output_df], ignore_index=True)

        # below we want the average word counts for questions
        # so we need a df with only the questions actually used here
        # i.e., the inner merge. output_df doesn't have the texts columns
        # so we'll do a second merge here real quick to facilitate that count
        texts_df = actual.merge(texts, on="ID")

        # only include grades for those questions defined above
        actual = actual[[x for x in actual.columns if x in exam["columns"]]]

        # merge the machine-scored dataframe with that of the actual grades
        df_exam = actual.merge(output_df, on="ID")
        
        # display a preview of the two scores (first few rows of the merged dataframe)
        # I'm excluding the ID here to avoid sharing IDs in the cell's output
        #display(df_exam[[x for x in list(df_exam.columns.values) if (x not in ["ID"])]].head())
        display(df_exam.head())

        print("Number of entries:",len(df_exam))

        # for each of the questions that were scored
        for qn in [x for x in exam["columns"] if (x not in ["ID"])]:
            mean_words = int(round(texts_df["size_%s"%qn].mean()))
            print("\n%s\nMean Words:"%qn,mean_words)

            # create a dataframe with the grade and machine scores for this question
            # drop any nans
            df = df_exam[["ID", "%s_x"%qn, "%s_y"%qn]].dropna()
            
            if score==2:
                # get ICC for score==2 (i.e., when both scores are z scores)
                df_icc_tmp = df_icc.copy()
                df_icc_tmp["score"] = df_icc_tmp[qn]
                df_icc_tmp = df_icc_tmp[["ID","judge","score"]]
                display(df_icc_tmp.head()) 
                icc = pg.intraclass_corr(data=df_icc_tmp, targets='ID', raters='judge', ratings='score',nan_policy='omit')
                display(icc.set_index('Type'))
                
                # Kappa isn't meant for continuous scoring, so we're binning the z score 
                # multipling by ten and rounding the product. 
                kappa = cohen_kappa_score(round(df["%s_x"%qn]*10), round(df["%s_y"%qn]*10),weights="quadratic")
                print("Cohen's kappa for rounded(z scores x 10):",kappa)

            # create an list of these scores then count the swaps needed to bring them into 
            # agreement with the grades, averaged over `runs`. Append this to our list of 
            # swaps needed to move from machine-scored ordering to that of the graded exams. 
            # we're shuffling before sorting to address the fact that the original order of
            # rows can effect the swaps as the order of actual scores could be different
            # within the bins provided by the machine grade 
            tmp = []
            for x in range(runs):
                # shuffle df then sort answers by the machine scores
                # this is needed becaue the `actual` rows can effect the number 
                # of swaps, so we want to get a represnative sample over `runs`
                # note the use of `sample.` This is where we shuffle the base df
                df = df.sample(frac=1).sort_values(by='%s_y'%qn, ascending=True)
                
                arr_ = df[df.columns[1]].values.copy()
                n = len(arr_)
                M_o = countSwaps(arr_, n)
                tmp.append(M_o)

            M_o = np.array(tmp).mean()
            M.append(M_o)

            # we're going to do the same as the above but for pseudo-random shuffles.
            # we'll shuffle and count swaps for the number of times defined in `runs`
            # and average the results 
            tmp = []
            for x in range(runs):
                # note the use of `sample.` This is where we get our pseudo-random ordering
                arr_ = df[df.columns[1]].sample(frac=1).values.copy()
                n = len(arr_)
                R_o = countSwaps(arr_, n)
                tmp.append(R_o)

            R_o = np.array(tmp).mean()
            R.append(R_o)
            
            print("Algo score, swaps needed:",M_o)
            print("Swaps needed for first 10 out of %s pseudo-random runs:"%runs)
            print(tmp[:10])
            print("Pseudo-random, avergae swaps needed:",R_o)
            
    print("\n=========================================================")
    print("Swaps (Machine):")
    print(M)
    print("N:",len(M),"\tMean: ",np.mean(M),"\tVar: ",np.var(M))
    print("Avergae swaps (Pseudo-random):")
    print(R)
    print("N:",len(R),"\tMean: ",np.mean(R),"\tVar: ",np.var(R),"\n")
    
    
    # Print out a summary table of the percent difference between pseudo-random and machine swaps
    pdiff = []
    i = 0
    for x in R:
        if x != 0 and M[i] != 0:
            pdiff_ = 100*(x-M[i])/((x+M[i])/2)
            pdiff.append(pdiff_)
            print( i+1,")",x,">",M[i],"=>",pdiff_ )
        else:
            print("error")
        i = i + 1

    print("\nAvergae percent difference between swaps needed for pseudo-random and machine sorting:")
    print(np.array(pdiff).mean(),"\n")
        
    # run t test for the two populations of swaps
    sample_n = len(M)
    print(stats.ttest_rel(M[:sample_n], R[:sample_n]))
    # return the effect size
    print("Cohen's d: ",cohend(R[:sample_n], M[:sample_n]))

In [4]:
def grade(df,model,normv,score=0,goal="centroid"):
       
    df = df.copy()    
    
    # exclude the ID column to get only the answer texts
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    # vectorize the text of each answer
    for x in questions:
        df["%s_vec"%x] = df[x].apply(vectorize, args=(model,normv)) 
    
    # for each question
    for x in questions:
        
        # based on the `goal` parameter grade, against either the centroid or medoid
        # that is, set either as the model answer aginst which to grade others
        if goal=="centroid":
            # centroid 
            best_ans = df["%s_vec"%x].mean()
        else:
            # medoid 
            best_ans = df["%s_vec"%x].iloc[best_answer(df,x)]

        # measure each answer's distance from the model answer above
        df["%s_dist"%x] = df["%s_vec"%x].apply(distance, args=(best_ans,))
        
        # replace the text of the answer with its z-score
        df[x] =  df["%s_vec"%x].apply(z_score, args=(df,x,best_ans))
        
        # translate the z-score into a standard grade using
        # the translation defined by `score`
        if score==0:
            df[x] =  df[x].apply(score_0)
        elif score==1:
            df[x] =  df[x].apply(score_1)
        elif score==2:
            print("Leaving z scores in place")
            
    # create output df without the vectors or distance measures
    output_df = df[[ x for x in list(df.columns.values) if ("_vec" not in x) and ("_dist" not in x) ]].copy()
            
    return output_df

In [5]:
def vectorize(row,model='nlp1',normv=0): 

    # use a method for vectorization based on the `model` parameter
    # note: some methods can work across the entire text of a qustion
    # others requier vectorization at the sentence level
    if model=='nlp1':
        nlp = nlp1
        mode = 0
    elif model=='nlp2':
        nlp = nlp2
        mode = 1
    elif model=='nlp3':
        nlp = nlp3
        mode = 1
        
    try:
        if mode == 1:
            # vectorize at the sentence level
            doc = nlp(row)
            tmp_df = pd.DataFrame({'sent' : []})
            i=0 
            for sent in doc.sents:
                sent_vec = nlp(sent.string.strip()).vector
                # if normv=1 normalize each sentence vector
                if normv==1:
                    sent_vec = norm(sent_vec)
                if i==0:
                    tmp_df = pd.DataFrame([[sent_vec]],columns=["sent"])
                else:
                    tmp_df = tmp_df.append(pd.DataFrame([[sent_vec]],columns=["sent"]), ignore_index=True)
                i+=1
            # average sentence vectors 
            output = tmp_df["sent"].mean()
        else:
            # vectorize all input text together
            output = nlp(row).vector

        # if normv=1 normalize the answer's vector
        if normv==1:
            output = norm(output)  
    
        return output
    except:
        return np.NaN
    
def norm(row):
    # normalize vector
    matrix = row.reshape(1,-1).astype(np.float64)
    return normalize(matrix, axis=1, norm='l1')[0]

In [6]:
def best_answer(df,column):
    # Return the medoid (like the centroid, but confined to a member of the group) 
    # i.e., the best answer in the group of answers.
    tmp_df = df[[ "%s_vec"%column ]].copy()
    centroid = tmp_df["%s_vec"%column].mean()
    tmp_df["%s_dist"%column] = tmp_df["%s_vec"%column].apply(distance, args=(centroid,))
    tmp_df = tmp_df[tmp_df["%s_dist"%column]==tmp_df["%s_dist"%column].min()]
    return tmp_df.index[0]

def distance(row,centroid):
    # calculate the Euclidean distance between centroid and row
    # See https://stackoverflow.com/a/1401828
    return np.linalg.norm(centroid-row)

In [7]:
def z_score(row,df,x,best):
    # calculate the z-score for a given distance
    return -1*(np.linalg.norm(best-row)-df["%s_dist"%x].mean())/df["%s_dist"%x].std()

def z_scores_from_grades(df):
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    for x in questions:
        df[x] = df[x].apply(z_score_tradional, args=(df,x))
    return df

def z_score_tradional(row,df,x):
    # calculate the z-score for a given score
    return ((row-df[x].mean())/df[x].std())

In [8]:
# Define grade bin boarders based on z-score
# --------------------------------------
# There's no standard mapping between z-scores and letter grades
# However, it's common for schools to set their own curve which 
# is in the same spirit. Therefor I have created a mapping where
# the average answer recieves a B. Others might choose differently

grade_bins = [
              2,    # A+
              1.5,  # A
              1,    # A-
              0.5,  # B+
              0,    # B
              -0.5, # B-
              -1,   # C+
              -1.5, # C
              -2,   # C-
              -2.5, # D+
              -3,   # D
              -3.5  # D-
             ]

# Assign numeric grades based on z-score
# -----------------------------------------
# This was the original z-score to percentage grade translation.
# I took the above and translated it into numberic grades
# with the scale bottoming out at 59 and topping out at 97
def score_0(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93+((row-grade_bins[1])/(grade_bins[0]-grade_bins[1]))*4
    elif row >= grade_bins[2]:
        grade = 90+((row-grade_bins[2])/(grade_bins[1]-grade_bins[2]))*3
    elif row >= grade_bins[3]:
        grade = 87+((row-grade_bins[3])/(grade_bins[2]-grade_bins[3]))*3
    elif row >= grade_bins[4]:
        grade = 83+((row-grade_bins[4])/(grade_bins[3]-grade_bins[4]))*4
    elif row >= grade_bins[5]:
        grade = 80+((row-grade_bins[5])/(grade_bins[4]-grade_bins[5]))*3
    elif row >= grade_bins[6]:
        grade = 77+((row-grade_bins[6])/(grade_bins[5]-grade_bins[6]))*3
    elif row >= grade_bins[7]:
        grade = 73+((row-grade_bins[7])/(grade_bins[6]-grade_bins[7]))*4
    elif row >= grade_bins[8]:
        grade = 70+((row-grade_bins[8])/(grade_bins[7]-grade_bins[8]))*3
    elif row >= grade_bins[9]:
        grade = 67+((row-grade_bins[9])/(grade_bins[8]-grade_bins[9]))*3
    elif row >= grade_bins[10]:
        grade = 63+((row-grade_bins[10])/(grade_bins[9]-grade_bins[10]))*4
    elif row >= grade_bins[11]:
        grade = 60+(row-grade_bins[11])*(3/(grade_bins[10]-grade_bins[11]))
    else:
        grade = 59
        
    # The final output is rounded to avoid false percision. A better 
    # approach is to bin all of the scores. This is done in score_1 below
    return round(grade)

# Assign numeric grades based on z-score
# -----------------------------------------
# Upon subsequent reflection, I realized that binned scores were preferable
# to a continuous score with dropoffs at 59 and 97. The above translation,
# however, was used on the first run of this code and so is included here
# for compleatness.  
def score_1(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93
    elif row >= grade_bins[2]:
        grade = 90
    elif row >= grade_bins[3]:
        grade = 87
    elif row >= grade_bins[4]:
        grade = 83
    elif row >= grade_bins[5]:
        grade = 80
    elif row >= grade_bins[6]:
        grade = 77
    elif row >= grade_bins[7]:
        grade = 73
    elif row >= grade_bins[8]:
        grade = 70
    elif row >= grade_bins[9]:
        grade = 67
    elif row >= grade_bins[10]:
        grade = 63
    elif row >= grade_bins[11]:
        grade = 60
    else:
        grade = 59
    return grade

In [9]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required 
# to sort an array when only swapping of adjacent 
# elements is allowed. 
# include <bits/stdc++.h> 

#This function merges two sorted arrays and returns inversion count in the arrays.*/ 
def merge(arr, temp, left, mid, right): 
    inv_count = 0

    i = left #i is index for left subarray*/ 
    j = mid #i is index for right subarray*/ 
    k = left #i is index for resultant merged subarray*/ 
    while ((i <= mid - 1) and (j <= right)): 
        if (arr[i] <= arr[j]): 
            temp[k] = arr[i] 
            k += 1
            i += 1
        else: 
            temp[k] = arr[j] 
            k += 1
            j += 1

            #this is tricky -- see above explanation/ 
            # diagram for merge()*/ 
            inv_count = inv_count + (mid - i) 

    #Copy the remaining elements of left subarray 
    # (if there are any) to temp*/ 
    while (i <= mid - 1): 
        temp[k] = arr[i] 
        k += 1
        i += 1

    #Copy the remaining elements of right subarray 
    # (if there are any) to temp*/ 
    while (j <= right): 
        temp[k] = arr[j] 
        k += 1
        j += 1

    # Copy back the merged elements to original array*/ 
    for i in range(left,right+1,1): 
        arr[i] = temp[i] 

    return inv_count 

#An auxiliary recursive function that sorts the input 
# array and returns the number of inversions in the 
# array. */ 
def _mergeSort(arr, temp, left, right): 
    inv_count = 0
    if (right > left): 
        # Divide the array into two parts and call 
        #_mergeSortAndCountInv() 
        # for each of the parts */ 
        mid = int((right + left)/2) 

        #Inversion count will be sum of inversions in 
        # left-part, right-part and number of inversions 
        # in merging */ 
        inv_count = _mergeSort(arr, temp, left, mid) 
        inv_count += _mergeSort(arr, temp, mid+1, right) 

        # Merge the two parts*/ 
        inv_count += merge(arr, temp, left, mid+1, right) 

    return inv_count 

#This function sorts the input array and returns the 
#number of inversions in the array */ 
def countSwaps(arr, n): 
    temp = [0 for i in range(n)] 
    return _mergeSort(arr, temp, 0, n - 1) 

In [10]:
# The following function calculates Cohen d (the effect size)
# based on two populations. It was written by Jason Brownlee
# See https://machinelearningmastery.com/effect-size-measures-in-python/
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

[back to contents](#Contents)

# Results

A subset of the data presented in the cell below can be found in Table 1 of _Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams_. 

In [11]:
score_exams(exams,model='nlp1',normv=1,score=2,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
Leaving z scores in place
Leaving z scores in place
Leaving z scores in place
Computing z scores for human scoring


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,f346f20d-4883-4fea-af93-f716f6e99274,1.521905,-0.147495,-0.386848,1.043495,0.453994,0.028075
1,ed56f2bc-52d8-4323-8f41-2cb80548a1dc,-0.159269,0.264623,0.650466,-0.487243,-1.628164,-0.336026
2,ddee1d35-ea9d-4e90-b6dd-4a9c5ff90aa0,-0.999856,-1.486880,-0.535036,0.636763,0.584401,0.496988
3,7b622388-4835-41b0-916e-bb4ca373d6c4,1.802101,0.264623,1.539592,0.835584,0.177698,0.284655
4,dcef7f09-f784-48cb-b65e-59d0293f4195,0.961514,0.676742,2.576906,0.462191,0.062287,0.393672


Number of entries: 81

SHORT_ANS
Mean Words: 436


,ID,judge,score
0,f346f20d-4883-4fea-af93-f716f6e99274,Human,1.521905
1,065476cf-e25b-4039-a5b0-69bfd27e61dd,Human,-0.999856
2,ed56f2bc-52d8-4323-8f41-2cb80548a1dc,Human,-0.159269
3,ddee1d35-ea9d-4e90-b6dd-4a9c5ff90aa0,Human,-0.999856
4,7b622388-4835-41b0-916e-bb4ca373d6c4,Human,1.802101


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.426259,2.485892,80,81,0.000030,"[0.23, 0.59]"
ICC2,Single random raters,0.424240,2.455869,80,80,0.000041,"[0.23, 0.59]"
ICC3,Single fixed raters,0.421274,2.455869,80,80,0.000041,"[0.22, 0.58]"
ICC1k,Average raters absolute,0.597730,2.485892,80,81,0.000030,"[0.38, 0.74]"
ICC2k,Average random raters,0.595742,2.455869,80,80,0.000041,"[0.37, 0.74]"
ICC3k,Average fixed raters,0.592812,2.455869,80,80,0.000041,"[0.37, 0.74]"


...

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.090017,1.197843,91,92,0.194626,"[-0.12, 0.29]"
ICC2,Single random raters,0.085672,1.185463,91,91,0.209323,"[-0.12, 0.29]"
ICC3,Single fixed raters,0.084862,1.185463,91,91,0.209323,"[-0.12, 0.28]"
ICC1k,Average raters absolute,0.165166,1.197843,91,92,0.194626,"[-0.26, 0.45]"
ICC2k,Average random raters,0.157823,1.185463,91,91,0.209323,"[-0.28, 0.44]"
ICC3k,Average fixed raters,0.156447,1.185463,91,91,0.209323,"[-0.28, 0.44]"


Cohen's kappa for rounded(z scores x 10): 0.15220449016453974
Algo score, swaps needed: 1705.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2177, 1981, 2158, 1882, 2158, 2116, 2215, 2154, 1748, 1832]
Pseudo-random, avergae swaps needed: 2045.71394

Q2
Mean Words: 2137


,ID,judge,score
0,69b6ce9c-4cdb-4afd-9ace-a993777b08e1,Human,-0.705991
1,e824f723-032e-4900-a6e5-a82e279eefc8,Human,-0.705991
2,b79bcda6-c021-453b-96e9-fd6d58c20469,Human,-1.284442
3,54273ba8-efd3-4d24-89aa-bb5f5cda0ee0,Human,0.258094
4,afb1f54f-6525-497c-9642-32aca074c609,Human,-1.284442


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.267120,1.728959,91,92,0.004700,"[0.07, 0.45]"
ICC2,Single random raters,0.264758,1.713932,91,91,0.005423,"[0.06, 0.45]"
ICC3,Single fixed raters,0.263062,1.713932,91,91,0.005423,"[0.06, 0.44]"
ICC1k,Average raters absolute,0.421617,1.728959,91,92,0.004700,"[0.13, 0.62]"
ICC2k,Average random raters,0.418669,1.713932,91,91,0.005423,"[0.12, 0.62]"
ICC3k,Average fixed raters,0.416546,1.713932,91,91,0.005423,"[0.12, 0.61]"


Cohen's kappa for rounded(z scores x 10): 0.25302929011688136
Algo score, swaps needed: 1589.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1864, 1885, 2070, 2055, 2154, 1972, 1830, 2085, 1889, 1932]
Pseudo-random, avergae swaps needed: 1994.7508

Swaps (Machine):
[998.0, 1054.0, 930.0, 1275.0, 1224.0, 117.0, 903.0, 830.99856, 1176.0, 1043.0, 783.0, 1705.0, 1589.0]
N: 13 	Mean:  1048.3075815384616 	Var:  141907.95346760284
Avergae swaps (Pseudo-random):
[1499.68528, 1566.90241, 1560.20248, 1838.09328, 1756.86311, 182.35202, 1323.80177, 1219.98775, 1472.10551, 1449.9545, 1457.60233, 2045.71394, 1994.7508]
N: 13 	Mean:  1489.8473215384618 	Var:  198489.0188468986 

1 ) 1499.68528 > 998.0 => 40.17201718865076
2 ) 1566.90241 > 1054.0 => 39.13937489950265
3 ) 1560.20248 > 930.0 => 50.61455725479801
4 ) 1838.09328 > 1275.0 => 36.17580517857146
5 ) 1756.86311 > 1224.0 => 35.75226975115942
6 ) 182.35202 > 117.0 => 43.66232103594958
7 ) 1323.80177 > 903.0 => 37.79427299449291
8 

The following cell presents the output for the parameters used when all of the exam data was first run through the above scoring method, with the exception of `runs`. To avoid biasing parameter selection, prior work on the method did NOT use the exam data used here. 

Note: the below cell has been re-run since this first run. Additionally, the current version makes use of the average number of swaps over 100,000 instances to provide a better representation of the method's performance. This is because the number of swaps can vary by small amounts for scoring methods where the granularity of the machine scores/grading scale is sufficiently coarse for exams with different human scores to find themselves with the same machine grades. That is, when sorting on the machine grade, one could get different absolute orderings based on the initial order of the list. For example: 

|ID|Q1_human|Q1_machine|
|--|--|--|
|00001|4|90|
|00002|5|90|

and

|ID|Q1_human|Q1_machine|
|--|--|--|
|00002|5|90|
|00001|4|90|

are both proper orderings based on the values of `Q1_machine` and could occur if the list before sorting had different orderings. In all cases below, the difference between swaps needed for the pseudo-random and machine orderings exhibit a significant difference.

Data from the cell above is cited in the paper so that comparisons can be made between the performance of machine, pseudo-random, and human orderings whereas the data in the cell below includes only comparisons between machine and pseudo-random orderings. 

In [12]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,351e0d0e-9901-4878-9d31-decad062eb91,2,21,6,90,87,81
1,0adab6a0-9008-4fdb-9fb7-a0be1edcda37,8,16,8,78,88,84
2,70d749d7-dd2e-4b83-be1a-1409b0dcb3e2,9,35,17,85,84,86
3,7a25c1e8-4bd9-438f-ad5b-393ad679a0ab,3,13,11,82,81,83
4,031928ff-6248-4987-bb30-75af9273ea92,10,41,21,83,85,85


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 989.50331
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1591, 1599, 1465, 1386, 1545, 1389, 1609, 1744, 1616, 1637]
Pseudo-random, avergae swaps needed: 1499.72726

Q1
Mean Words: 2048
Algo score, swaps needed: 1021.46179
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1688, 1418, 1513, 1558, 1453, 1742, 1635, 1270, 1463, 1713]
Pseudo-random, avergae swaps needed: 1566.17577

Q2
Mean Words: 947
Algo score, swaps needed: 829.93601
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1560, 1477, 1644, 1828, 1799, 1743, 1766, 1683, 1562, 1394]
Pseudo-random, avergae swaps needed: 1560.0728

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,ecdebb90-4db9-4967-b6f9-ea3759679fcc,13.0,16.0,90,94
1,d57e3e5c-10f4-4610-9ea6-49852423ede4,18.0,23.0,88,87
2,6f684adb-beb1-4e9c-8d7d-76e0637baf2b,14.0,18.0,85,84
3,d2a0868b-f1ef-43d6-a277-0b4b33bd78d0,11.5,21.0,82,79
4,da455e55-d2ac-4054-aae9-e62723a9dc6c,17.0,17.0,79,84


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1289.51587
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1774, 1743, 1938, 1834, 1669, 1834, 2189, 1843, 1991, 1652]
Pseudo-random, avergae swaps needed: 1837.38272

Q2
Mean Words: 1546
Algo score, swaps needed: 1319.45138
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1776, 1771, 1959, 1737, 1595, 2008, 1706, 1698, 1872, 1868]
Pseudo-random, avergae swaps needed: 1757.88976

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 908.03549
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1502, 1318, 1217, 1319, 1269, 1358, 1401, 1272, 1337, 1195]
Pseudo-random, avergae swaps needed: 1324.44786

Q2
Mean Words: 570
Algo score, swaps needed: 814.05907
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1264, 1108, 1349, 1480, 1260, 963, 1205, 1043, 1220, 998]
Pseudo-random, avergae swaps needed: 1219.65644

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,737af13d-d5ba-4cf8-beaa-63bdc403d760,38,2,17,85,79,82
1,6c715ed9-9520-4c0d-8d4f-b56ceef0320a,53,43,15,81,86,81
2,bb065bd5-206a-4c0f-ba0f-f193651f5ac6,62,23,40,83,84,88
3,cbe7586a-af37-49e3-86ad-4e36380adad9,74,40,45,90,91,90
4,496aeba0-ea0b-4003-bd5b-7e4ac8bd9073,47,35,20,86,78,81


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1176.01861
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1388, 1298, 1489, 1456, 1549, 1324, 1352, 1599, 1684, 1609]
Pseudo-random, avergae swaps needed: 1471.9416

Q2
Mean Words: 1050
Algo score, swaps needed: 995.51218
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1358, 1507, 1473, 1510, 1686, 1271, 1371, 1605, 1644, 1360]
Pseudo-random, avergae swaps needed: 1449.70805

Q3
Mean Words: 839
Algo score, swaps needed: 769.55054
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1440, 1391, 1549, 1429, 1392, 1520, 1326, 1485, 1404, 1589]
Pseudo-random, avergae swaps needed: 1457.44482

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,fc6ff039-d412-461c-a096-7039bac5bd73,25.25,20.5,76,62
1,d18b3687-1b51-4f7b-b1e4-94b2918c2e1f,16.00,17.0,82,89
2,230325ce-77e5-46d5-adf5-e088088aee8f,21.00,21.0,87,81
3,da2f8241-c0df-4081-b6bc-347b35a3d470,21.00,21.0,83,83
4,0c3b2e97-82af-40a0-81c9-0e19e9ddff09,19.00,20.0,90,91


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1777.99084
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2057, 1872, 1910, 1664, 2135, 2271, 2014, 1972, 2058, 1751]
Pseudo-random, avergae swaps needed: 2045.42864

Q2
Mean Words: 2137
Algo score, swaps needed: 1635.99287
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2278, 1949, 1648, 2197, 1839, 1946, 1976, 2155, 2021, 2084]
Pseudo-random, avergae swaps needed: 1994.22284

Swaps (Machine):
[989.50331, 1021.46179, 829.93601, 1289.51587, 1319.45138, 121.01466, 908.03549, 814.05907, 1176.01861, 995.51218, 769.55054, 1777.99084, 1635.99287]
N: 13 	Mean:  1049.8494323076923 	Var:  160966.89750620702
Avergae swaps (Pseudo-random):
[1499.72726, 1566.17577, 1560.0728, 1837.38272, 1757.88976, 182.61788, 1324.44786, 1219.65644, 1471.9416, 1449.70805, 1457.44482, 2045.42864, 1994.22284]
N: 13 	Mean:  1489.7474184615385 	Var:  198364.50812390135 

1 ) 1499.72726 > 989.50331 => 40.994511006668205
2 

The following cells present a variety of parameter selections, including a mix of vector normalization, grade translations, and "best answer" selections.

In [13]:
score_exams(exams,model='nlp1',normv=1,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,a503bcd8-7709-45ef-b48f-d56131f99108,6,15,11,78,84,86
1,dcef7f09-f784-48cb-b65e-59d0293f4195,10,30,32,87,83,86
2,ccd4c7f7-60a4-48c3-9dbf-96e923a98d93,6,23,28,86,79,86
3,c74cd4a8-869a-4714-892e-be61183e7835,4,27,24,89,91,83
4,7a4dfc2e-553b-4bc6-bc18-3b0626c17a1e,5,18,18,85,88,84


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 996.46505
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1310, 1488, 1320, 1562, 1612, 1426, 1638, 1551, 1372, 1488]
Pseudo-random, avergae swaps needed: 1499.14618

Q1
Mean Words: 2048
Algo score, swaps needed: 1054.98323
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1658, 1752, 1703, 1683, 1707, 1709, 1600, 1737, 1774, 1408]
Pseudo-random, avergae swaps needed: 1566.48594

Q2
Mean Words: 947
Algo score, swaps needed: 959.91972
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1471, 1484, 1446, 1407, 1472, 1561, 1721, 1494, 1540, 1534]
Pseudo-random, avergae swaps needed: 1560.00533

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,8c9de0d1-eff5-4b5c-b1ff-7cecd2bb6c34,12.0,12.0,65,80
1,1b15cca5-b292-4c29-8e83-384ed76e7b77,20.0,18.0,79,83
2,d9863d9e-0473-482d-ae21-bcfb05f1753c,9.0,17.0,71,82
3,a9738fe5-2df1-4661-b147-de435b7eeeb6,17.0,20.0,80,94
4,529faf72-9186-4652-8fcc-58ba8b46a5cb,22.0,20.0,89,86


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1247.49193
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1813, 1981, 1568, 2121, 1575, 1863, 2028, 1707, 1575, 1886]
Pseudo-random, avergae swaps needed: 1837.51867

Q2
Mean Words: 1546
Algo score, swaps needed: 1213.96858
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1938, 1632, 1899, 1795, 1660, 2098, 1546, 1900, 1834, 1816]
Pseudo-random, avergae swaps needed: 1758.20714

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 908.98728
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1473, 1182, 1417, 1249, 1410, 1401, 1076, 1334, 1079, 1336]
Pseudo-random, avergae swaps needed: 1324.01157

Q2
Mean Words: 570
Algo score, swaps needed: 907.39679
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1154, 1370, 1052, 1319, 1287, 1226, 1149, 1108, 1222, 1072]
Pseudo-random, avergae swaps needed: 1219.49946

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,496aeba0-ea0b-4003-bd5b-7e4ac8bd9073,47,35,20,83,74,81
1,49fdf82b-9794-42a5-9a7b-8fb177111688,28,25,17,80,82,80
2,ed6cf722-94b9-4474-8bf7-318cb15713b8,67,25,28,85,89,87
3,c1b34a08-528e-4824-85ac-1d2043c2bb32,15,5,2,80,81,78
4,ee5b756f-84b7-4707-ab9a-a9064b65ebe2,43,25,16,87,85,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1174.50544
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1373, 1263, 1463, 1603, 1533, 1465, 1509, 1349, 1563, 1521]
Pseudo-random, avergae swaps needed: 1472.93425

Q2
Mean Words: 1050
Algo score, swaps needed: 1041.00029
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1430, 1487, 1436, 1619, 1536, 1363, 1301, 1294, 1438, 1435]
Pseudo-random, avergae swaps needed: 1450.00772

Q3
Mean Words: 839
Algo score, swaps needed: 803.94188
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1520, 1399, 1458, 1504, 1481, 1406, 1533, 1621, 1241, 1440]
Pseudo-random, avergae swaps needed: 1458.14977

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,0a1cbcb8-fb01-45be-bb70-9c97770f647c,18.0,11.0,83,84
1,25b1719f-d684-490a-b15d-9596e54c29ee,14.0,3.5,76,63
2,5b092aa8-350b-4919-8267-f3309dbcb8fc,19.5,19.0,81,81
3,348bfa7e-1c58-4b5b-9b11-af9b507ee3cb,17.0,15.0,79,84
4,858981e3-e43d-4777-ac97-16678c7f6ac0,19.0,20.0,88,92


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1688.05055
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2388, 1906, 2017, 2148, 2162, 2038, 2254, 1994, 2205, 1959]
Pseudo-random, avergae swaps needed: 2045.85193

Q2
Mean Words: 2137
Algo score, swaps needed: 1578.9718
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2294, 1959, 1827, 1962, 2017, 2153, 2094, 1988, 1979, 1939]
Pseudo-random, avergae swaps needed: 1993.83577

Swaps (Machine):
[996.46505, 1054.98323, 959.91972, 1247.49193, 1213.96858, 114.0029, 908.98728, 907.39679, 1174.50544, 1041.00029, 803.94188, 1688.05055, 1578.9718]
N: 13 	Mean:  1053.0527261538462 	Var:  135086.9684930775
Avergae swaps (Pseudo-random):
[1499.14618, 1566.48594, 1560.00533, 1837.51867, 1758.20714, 182.47244, 1324.01157, 1219.49946, 1472.93425, 1450.00772, 1458.14977, 2045.85193, 1993.83577]
N: 13 	Mean:  1489.8558592307693 	Var:  198432.03504219826 

1 ) 1499.14618 > 996.46505 => 40.28521141091355
2 ) 

In [14]:
score_exams(exams,model='nlp1',normv=0,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,3563df1a-2ee1-4323-a204-fb3a13e13f57,1,1,4,80,70,77
1,70d749d7-dd2e-4b83-be1a-1409b0dcb3e2,9,35,17,83,83,83
2,354b8e2c-ce96-49fc-8a7b-80c7b8f04be3,3,13,2,80,63,80
3,e8c7236a-2cae-40ea-8d17-5405ce4a0e0f,12,17,14,87,80,80
4,07ac7940-e702-498a-a97c-333cb233a847,2,14,4,77,83,77


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 991.10428
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1310, 1608, 1730, 1663, 1360, 1523, 1626, 1454, 1405, 1740]
Pseudo-random, avergae swaps needed: 1500.25208

Q1
Mean Words: 2048
Algo score, swaps needed: 1023.11096
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1431, 1541, 1747, 1377, 1676, 1525, 1692, 1442, 1375, 1642]
Pseudo-random, avergae swaps needed: 1566.50075

Q2
Mean Words: 947
Algo score, swaps needed: 918.7735
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1702, 1558, 1459, 1443, 1767, 1453, 1580, 1413, 1650, 1367]
Pseudo-random, avergae swaps needed: 1560.09365

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,05ab5f67-d7d3-441f-8d3c-e879ece151f9,19.0,22.0,80,80
1,d3d89043-c41a-4cf9-aba0-17b47f4750c9,14.0,12.0,87,87
2,6dfac62a-26c6-4c69-afe0-39aa1b1a3e44,15.0,19.0,83,83
3,d57e3e5c-10f4-4610-9ea6-49852423ede4,18.0,23.0,87,87
4,6c430619-e918-45cc-8164-b092cb27ee8a,11.0,21.0,83,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1331.50755
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1666, 1787, 1725, 1750, 1861, 1565, 2026, 1805, 1799, 2077]
Pseudo-random, avergae swaps needed: 1837.78548

Q2
Mean Words: 1546
Algo score, swaps needed: 1371.00386
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1762, 1976, 1553, 1693, 1799, 1760, 1792, 1938, 1873, 1643]
Pseudo-random, avergae swaps needed: 1757.7674

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 903.94001
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1455, 1467, 1342, 1351, 1419, 1302, 1333, 1366, 1257, 1313]
Pseudo-random, avergae swaps needed: 1324.79922

Q2
Mean Words: 570
Algo score, swaps needed: 1074.27863
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1257, 1232, 1104, 1248, 1359, 1134, 1220, 1170, 1187, 1202]
Pseudo-random, avergae swaps needed: 1218.77426

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,e4873808-277a-47f8-9235-2188343dc4ab,32,33,30,87,87,83
1,496aeba0-ea0b-4003-bd5b-7e4ac8bd9073,47,35,20,83,77,80
2,737af13d-d5ba-4cf8-beaa-63bdc403d760,38,2,17,83,77,80
3,1dd97d65-c89a-4744-bfc1-c14311cc97b7,38,36,20,80,83,80
4,7e5c12f6-95ca-485d-91ec-71a4031f22d5,20,5,20,80,77,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1189.94081
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1534, 1358, 1507, 1549, 1670, 1694, 1405, 1519, 1365, 1353]
Pseudo-random, avergae swaps needed: 1472.50809

Q2
Mean Words: 1050
Algo score, swaps needed: 1004.40795
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1523, 1338, 1593, 1306, 1328, 1397, 1350, 1304, 1455, 1503]
Pseudo-random, avergae swaps needed: 1450.15038

Q3
Mean Words: 839
Algo score, swaps needed: 773.30653
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1637, 1310, 1430, 1499, 1580, 1559, 1270, 1730, 1328, 1565]
Pseudo-random, avergae swaps needed: 1458.04045

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,95eca05c-7d2d-4e74-80e7-f4dd862e71b1,24.5,24.0,87,87
1,c268db25-0c25-4dcb-a5dd-13e0b0b3b4e5,26.5,21.0,83,87
2,d5556a5d-e8c0-4613-853e-e6448f546a32,20.5,18.0,87,87
3,d7cbd32b-0f8a-4ee7-a4a8-5679eab81ae5,11.5,13.0,80,83
4,c2f90f17-5912-4155-a6d4-60da6b5733b4,17.0,18.0,80,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1767.51708
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1877, 2270, 1927, 2161, 2495, 1985, 2122, 2196, 1972, 1813]
Pseudo-random, avergae swaps needed: 2045.88172

Q2
Mean Words: 2137
Algo score, swaps needed: 1633.86401
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2064, 1826, 1848, 1903, 1798, 2116, 1994, 1746, 1985, 2002]
Pseudo-random, avergae swaps needed: 1993.96205

Swaps (Machine):
[991.10428, 1023.11096, 918.7735, 1331.50755, 1371.00386, 122.00948, 903.94001, 1074.27863, 1189.94081, 1004.40795, 773.30653, 1767.51708, 1633.86401]
N: 13 	Mean:  1084.981896153846 	Var:  155756.32549119272
Avergae swaps (Pseudo-random):
[1500.25208, 1566.50075, 1560.09365, 1837.78548, 1757.7674, 182.51655, 1324.79922, 1218.77426, 1472.50809, 1450.15038, 1458.04045, 2045.88172, 1993.96205]
N: 13 	Mean:  1489.925544615385 	Var:  198445.37818547385 

1 ) 1500.25208 > 991.10428 => 40.87314108688972
2 )

In [15]:
score_exams(exams,model='nlp1',normv=1,score=2,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
Leaving z scores in place
Leaving z scores in place
Leaving z scores in place
Computing z scores for human scoring


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,68975dd8-703c-43ba-8247-1e6cde4fe067,0.120927,-0.147495,0.798654,0.503427,0.869034,0.537341
1,72162df3-a93a-447b-9155-24b29b7c4a05,0.401122,0.367653,0.354091,0.530529,0.201332,0.047264
2,ddee1d35-ea9d-4e90-b6dd-4a9c5ff90aa0,-0.999856,-1.486880,-0.535036,0.636763,0.584401,0.496988
3,b54398fb-14ce-4ad0-9a86-9813160d627f,2.362492,2.119157,1.391404,0.516124,0.262030,0.379471
4,02694d63-a2bc-45b7-98fd-77c89066e667,-1.280052,-1.074762,-0.535036,0.007574,-1.101450,0.458851


Number of entries: 81

SHORT_ANS
Mean Words: 436


,ID,judge,score
0,68975dd8-703c-43ba-8247-1e6cde4fe067,Human,0.120927
1,72162df3-a93a-447b-9155-24b29b7c4a05,Human,0.401122
2,ddee1d35-ea9d-4e90-b6dd-4a9c5ff90aa0,Human,-0.999856
3,b54398fb-14ce-4ad0-9a86-9813160d627f,Human,2.362492
4,02694d63-a2bc-45b7-98fd-77c89066e667,Human,-1.280052


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.426259,2.485892,80,81,0.000030,"[0.23, 0.59]"
ICC2,Single random raters,0.424240,2.455869,80,80,0.000041,"[0.23, 0.59]"
ICC3,Single fixed raters,0.421274,2.455869,80,80,0.000041,"[0.22, 0.58]"
ICC1k,Average raters absolute,0.597730,2.485892,80,81,0.000030,"[0.38, 0.74]"
ICC2k,Average random raters,0.595742,2.455869,80,80,0.000041,"[0.37, 0.74]"
ICC3k,Average fixed raters,0.592812,2.455869,80,80,0.000041,"[0.37, 0.74]"


...

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.090017,1.197843,91,92,0.194626,"[-0.12, 0.29]"
ICC2,Single random raters,0.085672,1.185463,91,91,0.209323,"[-0.12, 0.29]"
ICC3,Single fixed raters,0.084862,1.185463,91,91,0.209323,"[-0.12, 0.28]"
ICC1k,Average raters absolute,0.165166,1.197843,91,92,0.194626,"[-0.26, 0.45]"
ICC2k,Average random raters,0.157823,1.185463,91,91,0.209323,"[-0.28, 0.44]"
ICC3k,Average fixed raters,0.156447,1.185463,91,91,0.209323,"[-0.28, 0.44]"


Cohen's kappa for rounded(z scores x 10): 0.15220449016453974
Algo score, swaps needed: 1705.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2081, 2187, 2193, 2140, 2030, 1831, 2220, 1940, 2074, 1901]
Pseudo-random, avergae swaps needed: 2045.00169

Q2
Mean Words: 2137


,ID,judge,score
0,1345e6e8-c15f-4a31-a12a-3642a8d2eb92,Human,0.836545
1,b79bcda6-c021-453b-96e9-fd6d58c20469,Human,-1.284442
2,85ca12a0-f4e5-4045-8350-676e0e159221,Human,0.258094
3,da2f8241-c0df-4081-b6bc-347b35a3d470,Human,1.029362
4,c2f90f17-5912-4155-a6d4-60da6b5733b4,Human,0.450911


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.267120,1.728959,91,92,0.004700,"[0.07, 0.45]"
ICC2,Single random raters,0.264758,1.713932,91,91,0.005423,"[0.06, 0.45]"
ICC3,Single fixed raters,0.263062,1.713932,91,91,0.005423,"[0.06, 0.44]"
ICC1k,Average raters absolute,0.421617,1.728959,91,92,0.004700,"[0.13, 0.62]"
ICC2k,Average random raters,0.418669,1.713932,91,91,0.005423,"[0.12, 0.62]"
ICC3k,Average fixed raters,0.416546,1.713932,91,91,0.005423,"[0.12, 0.61]"


Cohen's kappa for rounded(z scores x 10): 0.25302929011688136
Algo score, swaps needed: 1589.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2053, 1928, 1861, 2327, 2038, 1984, 1850, 1868, 1715, 2068]
Pseudo-random, avergae swaps needed: 1994.42136

Swaps (Machine):
[998.0, 1054.0, 930.0, 1275.0, 1224.0, 117.0, 903.0, 831.00127, 1176.0, 1043.0, 783.0, 1705.0, 1589.0]
N: 13 	Mean:  1048.30779 	Var:  141907.86286697842
Avergae swaps (Pseudo-random):
[1499.90568, 1566.23236, 1560.29005, 1837.85846, 1757.2092, 182.56658, 1323.97221, 1219.08912, 1472.53206, 1449.32702, 1457.98898, 2045.00169, 1994.42136]
N: 13 	Mean:  1489.722674615385 	Var:  198388.26468606733 

1 ) 1499.90568 > 998.0 => 40.186119437464114
2 ) 1566.23236 > 1054.0 => 39.09823936377917
3 ) 1560.29005 > 930.0 => 50.61981033092913
4 ) 1837.85846 > 1275.0 => 36.16344702033127
5 ) 1757.2092 > 1224.0 => 35.77133734861679
6 ) 182.56658 > 117.0 => 43.77429551721022
7 ) 1323.97221 > 903.0 => 37.806687313803515
8 ) 121

In [16]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,499fc744-a3e7-4927-afd5-b4df47ca7874,0,25,10,73,87,80
1,ebd5efa9-f2f3-47f8-9959-1918046163c8,4,13,8,87,83,83
2,341a81d8-db2d-4017-95ca-f11d39c62331,5,10,10,87,83,83
3,7b622388-4835-41b0-916e-bb4ca373d6c4,13,26,25,83,83,80
4,6e472a30-2be4-4670-bff5-0a78a5fab822,9,33,21,70,87,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1441.96402
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1436, 1483, 1738, 1483, 1603, 1516, 1292, 1531, 1522, 1636]
Pseudo-random, avergae swaps needed: 1499.1106

Q1
Mean Words: 2048
Algo score, swaps needed: 1165.3767
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1689, 1696, 1601, 1684, 1452, 1535, 1388, 1478, 1512, 1470]
Pseudo-random, avergae swaps needed: 1566.27993

Q2
Mean Words: 947
Algo score, swaps needed: 1284.64089
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1535, 1616, 1611, 1268, 1311, 1636, 1582, 1632, 1591, 1562]
Pseudo-random, avergae swaps needed: 1560.16865

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,8d5c28dd-7532-4b93-b654-7c254bd47fce,17.0,20.0,87,90
1,da455e55-d2ac-4054-aae9-e62723a9dc6c,17.0,17.0,73,80
2,ebf4deeb-44b2-4c4d-992e-b4e44cda8cc9,17.5,20.0,90,83
3,af76b772-5886-4e53-a62d-e87b17f052c9,13.0,15.0,70,77
4,faca6a15-3d4c-4771-ad55-f8fc6f55b71a,10.0,13.0,73,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1217.07259
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1564, 1857, 1897, 2003, 1973, 1738, 1768, 1815, 1870, 1777]
Pseudo-random, avergae swaps needed: 1837.17634

Q2
Mean Words: 1546
Algo score, swaps needed: 1703.51235
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1900, 1846, 1666, 1523, 1645, 1857, 1991, 1795, 1687, 1664]
Pseudo-random, avergae swaps needed: 1757.39543

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 1024.01478
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1384, 1308, 1303, 1302, 1298, 1362, 1207, 1375, 1230, 1514]
Pseudo-random, avergae swaps needed: 1324.45789

Q2
Mean Words: 570
Algo score, swaps needed: 1129.23608
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1216, 1280, 1276, 1289, 1081, 1243, 1201, 1239, 1133, 1292]
Pseudo-random, avergae swaps needed: 1219.24489

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,b21e248a-45c5-43ae-b06a-10261395fd1c,66,28,33,87,77,80
1,1b273341-a255-40ef-9a90-17fbdf05af4b,48,39,31,80,83,87
2,a7bf4849-6187-42da-8439-600dce189d0f,37,28,19,73,83,83
3,30234d01-ef3f-41bf-ae4a-1efaba3522d8,72,41,23,83,87,87
4,fbc35066-d18b-49cc-9540-9ec417e0e160,38,25,31,87,80,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1279.13438
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1659, 1575, 1460, 1316, 1404, 1372, 1392, 1448, 1530, 1249]
Pseudo-random, avergae swaps needed: 1472.41306

Q2
Mean Words: 1050
Algo score, swaps needed: 1111.52218
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1551, 1425, 1617, 1418, 1379, 1218, 1516, 1339, 1546, 1557]
Pseudo-random, avergae swaps needed: 1449.40899

Q3
Mean Words: 839
Algo score, swaps needed: 947.0725
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1365, 1473, 1398, 1606, 1517, 1234, 1233, 1528, 1500, 1492]
Pseudo-random, avergae swaps needed: 1457.26523

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,f234936b-f804-47e5-b956-9f3c9feef83c,20.5,22.0,83,87
1,83c20b41-645b-4931-97bf-870d498e43de,14.0,8.0,80,83
2,0091731f-bae9-4326-a66a-a744137584a2,13.5,8.0,83,77
3,858981e3-e43d-4777-ac97-16678c7f6ac0,19.0,20.0,90,90
4,85ca12a0-f4e5-4045-8350-676e0e159221,22.0,17.0,80,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 2023.38838
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2152, 2110, 1868, 2089, 2156, 2139, 1961, 1961, 2235, 1950]
Pseudo-random, avergae swaps needed: 2045.98698

Q2
Mean Words: 2137
Algo score, swaps needed: 1759.92724
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2193, 1875, 1769, 2061, 1820, 1979, 1942, 2009, 1967, 1817]
Pseudo-random, avergae swaps needed: 1993.77345

Swaps (Machine):
[1441.96402, 1165.3767, 1284.64089, 1217.07259, 1703.51235, 131.48975, 1024.01478, 1129.23608, 1279.13438, 1111.52218, 947.0725, 2023.38838, 1759.92724]
N: 13 	Mean:  1247.5655261538461 	Var:  195273.5144029216
Avergae swaps (Pseudo-random):
[1499.1106, 1566.27993, 1560.16865, 1837.17634, 1757.39543, 182.60821, 1324.45789, 1219.24489, 1472.41306, 1449.40899, 1457.26523, 2045.98698, 1993.77345]
N: 13 	Mean:  1489.6376653846155 	Var:  198367.60072464152 

1 ) 1499.1106 > 1441.96402 => 3.886102012603821


In [17]:
score_exams(exams,model='nlp1',normv=1,score=2,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
Leaving z scores in place
Leaving z scores in place
Leaving z scores in place
Computing z scores for human scoring


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,02694d63-a2bc-45b7-98fd-77c89066e667,-1.280052,-1.074762,-0.535036,1.229655,-1.120584,0.033619
1,8b2e671a-8164-4467-9f19-a6dc7cbf313c,-0.719661,0.367653,0.205903,0.555071,0.557392,0.432039
2,07ac7940-e702-498a-a97c-333cb233a847,-1.280052,-0.971732,-1.572349,-0.494572,0.849569,-0.376901
3,499fc744-a3e7-4927-afd5-b4df47ca7874,-1.840443,0.161594,-0.683223,-1.526789,1.024490,0.133701
4,0adab6a0-9008-4fdb-9fb7-a0be1edcda37,0.401122,-0.765673,-0.979599,-1.051691,0.474231,0.012859


Number of entries: 81

SHORT_ANS
Mean Words: 436


,ID,judge,score
0,02694d63-a2bc-45b7-98fd-77c89066e667,Human,-1.280052
1,8b2e671a-8164-4467-9f19-a6dc7cbf313c,Human,-0.719661
2,7ab0ea89-c22f-4ca6-bd74-c3aec57452f8,Human,-0.999856
3,07ac7940-e702-498a-a97c-333cb233a847,Human,-1.280052
4,499fc744-a3e7-4927-afd5-b4df47ca7874,Human,-1.840443


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.137978,1.320126,80,81,0.107329,"[-0.08, 0.34]"
ICC2,Single random raters,0.133377,1.304064,80,80,0.118631,"[-0.09, 0.34]"
ICC3,Single fixed raters,0.131969,1.304064,80,80,0.118631,"[-0.09, 0.34]"
ICC1k,Average raters absolute,0.242496,1.320126,80,81,0.107329,"[-0.18, 0.51]"
ICC2k,Average random raters,0.235362,1.304064,80,80,0.118631,"[-0.2, 0.51]"
ICC3k,Average fixed raters,0.233167,1.304064,80,80,0.118631,"[-0.19, 0.51]"


...

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.038908,1.080967,91,92,0.355056,"[-0.17, 0.24]"
ICC2,Single random raters,0.034047,1.069763,91,91,0.374203,"[-0.17, 0.24]"
ICC3,Single fixed raters,0.033706,1.069763,91,91,0.374203,"[-0.17, 0.24]"
ICC1k,Average raters absolute,0.074902,1.080967,91,92,0.355056,"[-0.4, 0.39]"
ICC2k,Average random raters,0.065852,1.069763,91,91,0.374203,"[-0.42, 0.38]"
ICC3k,Average fixed raters,0.065214,1.069763,91,91,0.374203,"[-0.41, 0.38]"


Cohen's kappa for rounded(z scores x 10): 0.1003820015488901
Algo score, swaps needed: 1757.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2043, 2059, 1988, 2385, 2113, 2180, 1834, 2101, 1961, 2254]
Pseudo-random, avergae swaps needed: 2045.23351

Q2
Mean Words: 2137


,ID,judge,score
0,59b5388b-5b2b-430c-a246-0d8b83249966,Human,0.836545
1,e2459d8b-2f3f-469e-b9f9-f5acfe633b0d,Human,1.800630
2,0ab76566-bd98-4796-b32d-f23794084401,Human,-0.127540
3,af5f9a64-909b-4e1b-8100-e82f9813b148,Human,1.414996
4,f0166970-fdaf-4f6b-9ae6-e8e25ef8b198,Human,-0.513174


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.095113,1.210220,91,92,0.181434,"[-0.11, 0.29]"
ICC2,Single random raters,0.091335,1.199202,91,91,0.193938,"[-0.12, 0.29]"
ICC3,Single fixed raters,0.090579,1.199202,91,91,0.193938,"[-0.12, 0.29]"
ICC1k,Average raters absolute,0.173704,1.210220,91,92,0.181434,"[-0.25, 0.45]"
ICC2k,Average random raters,0.167383,1.199202,91,91,0.193938,"[-0.26, 0.45]"
ICC3k,Average fixed raters,0.166112,1.199202,91,91,0.193938,"[-0.26, 0.45]"


Cohen's kappa for rounded(z scores x 10): 0.14589349325298806
Algo score, swaps needed: 1715.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2004, 1996, 1802, 2049, 2284, 1970, 1738, 2122, 2075, 1794]
Pseudo-random, avergae swaps needed: 1995.11491

Swaps (Machine):
[1320.0, 1231.0, 976.0, 1205.0, 1235.0, 191.0, 1107.0, 1026.9973, 1395.0, 1176.0, 942.0, 1757.0, 1715.0]
N: 13 	Mean:  1175.1536384615383 	Var:  138158.49941116854
Avergae swaps (Pseudo-random):
[1498.90842, 1566.7739, 1559.69847, 1838.54332, 1756.92053, 182.4611, 1323.96503, 1219.80319, 1472.55787, 1450.1969, 1458.46293, 2045.23351, 1995.11491]
N: 13 	Mean:  1489.8953907692307 	Var:  198469.2821672779 

1 ) 1498.90842 > 1320.0 => 12.693453872474509
2 ) 1566.7739 > 1231.0 => 24.002933189132968
3 ) 1559.69847 > 976.0 => 46.03847633350506
4 ) 1838.54332 > 1205.0 => 41.63195679435902
5 ) 1756.92053 > 1235.0 => 34.88866263436483
6 ) 182.4611 > 191.0 => -4.57284573948934
7 ) 1323.96503 > 1107.0 => 17.8501152688321

In [18]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,e021f764-a5d2-432a-89f5-00ba2b760f0f,9,31,23,87,87,87
1,4eb794d7-21bd-4bcd-96e7-6807a2197049,5,8,5,77,73,80
2,7a25c1e8-4bd9-438f-ad5b-393ad679a0ab,3,13,11,70,83,80
3,b8efdf86-6377-4fc4-b0b3-64d1bf7df43a,5,27,18,80,80,87
4,351e0d0e-9901-4878-9d31-decad062eb91,2,21,6,87,90,70


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1002.03083
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1519, 1547, 1536, 1539, 1672, 1510, 1401, 1408, 1372, 1525]
Pseudo-random, avergae swaps needed: 1498.81278

Q1
Mean Words: 2048
Algo score, swaps needed: 1193.03933
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1555, 1660, 1662, 1749, 1595, 1720, 1584, 1280, 1739, 1650]
Pseudo-random, avergae swaps needed: 1566.86461

Q2
Mean Words: 947
Algo score, swaps needed: 970.57382
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1492, 1512, 1656, 1665, 1685, 1666, 1411, 1295, 1426, 1706]
Pseudo-random, avergae swaps needed: 1560.04081

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,68202246-13a0-4c1f-a238-2590685176fa,20.0,21.0,90,87
1,d6db68eb-a97f-49b6-b974-8146d81c7417,13.5,13.0,80,80
2,d14fe6dc-ad38-4146-815b-82d3e66692cf,19.5,23.0,87,83
3,165f436c-ce5e-4cf6-8005-5c96b0801ab8,18.0,21.0,83,83
4,a6f29834-7a4e-4947-a2f0-3daeb1b23284,18.0,20.0,80,83


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1283.03442
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1884, 1924, 1945, 1884, 1731, 1831, 1816, 1727, 1834, 1888]
Pseudo-random, avergae swaps needed: 1837.85914

Q2
Mean Words: 1546
Algo score, swaps needed: 1230.06142
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2108, 1767, 1512, 1821, 1741, 1660, 1653, 1618, 1630, 1764]
Pseudo-random, avergae swaps needed: 1757.28153

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 919.00724
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1498, 1277, 1260, 1280, 1377, 1230, 1555, 1374, 1264, 1274]
Pseudo-random, avergae swaps needed: 1324.88911

Q2
Mean Words: 570
Algo score, swaps needed: 834.03755
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1213, 1156, 1373, 1219, 1103, 1335, 1077, 1069, 1339, 1345]
Pseudo-random, avergae swaps needed: 1219.81396

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,ba674cb4-50fb-4023-8bb7-5ae68452d116,55,40,41,83,93,87
1,ab397bfe-7c68-4f02-baee-46407830b974,36,28,22,87,87,83
2,11db58c4-af5b-4ffb-a2be-a807e6d3b0f6,34,26,22,83,87,80
3,09e22550-57f9-46d9-b5e2-63a3b0f30890,43,31,23,83,83,83
4,c013d719-f4b0-4788-8955-41e80cd92edf,41,32,41,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1139.95249
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1473, 1248, 1600, 1508, 1557, 1409, 1458, 1547, 1574, 1450]
Pseudo-random, avergae swaps needed: 1471.74111

Q2
Mean Words: 1050
Algo score, swaps needed: 908.52944
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1438, 1543, 1439, 1637, 1741, 1580, 1491, 1531, 1358, 1271]
Pseudo-random, avergae swaps needed: 1450.04051

Q3
Mean Words: 839
Algo score, swaps needed: 857.42491
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1573, 1620, 1499, 1287, 1396, 1525, 1296, 1598, 1455, 1386]
Pseudo-random, avergae swaps needed: 1458.2128

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,5588c2b6-6cba-4955-a41c-b54a085d6751,15.50,15.0,83,80
1,922969bb-2109-4d45-8d53-919f843faa07,23.00,16.0,77,83
2,19504cf2-0e8e-47e7-8cd8-37ccc42cc860,20.50,15.0,83,83
3,59b5388b-5b2b-430c-a246-0d8b83249966,19.00,20.0,83,80
4,f9b1bd52-ca9b-4c23-8758-7dcb7006a1b1,6.25,12.0,77,77


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1745.88055
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2349, 1807, 2094, 2042, 2084, 1777, 1822, 2271, 1952, 2124]
Pseudo-random, avergae swaps needed: 2045.07331

Q2
Mean Words: 2137
Algo score, swaps needed: 1731.0502
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1862, 1824, 1812, 1953, 1979, 1899, 1893, 2309, 1853, 1980]
Pseudo-random, avergae swaps needed: 1993.76677

Swaps (Machine):
[1002.03083, 1193.03933, 970.57382, 1283.03442, 1230.06142, 91.00509, 919.00724, 834.03755, 1139.95249, 908.52944, 857.42491, 1745.88055, 1731.0502]
N: 13 	Mean:  1069.6636376923077 	Var:  162116.16715370378
Avergae swaps (Pseudo-random):
[1498.81278, 1566.86461, 1560.04081, 1837.85914, 1757.28153, 182.40981, 1324.88911, 1219.81396, 1471.74111, 1450.04051, 1458.2128, 2045.07331, 1993.76677]
N: 13 	Mean:  1489.7543269230769 	Var:  198324.48312159677 

1 ) 1498.81278 > 1002.03083 => 39.729149636829945
2 

In [19]:
score_exams(exams,model='nlp2',normv=1,score=2,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
Leaving z scores in place
Leaving z scores in place
Leaving z scores in place
Computing z scores for human scoring


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7a4dfc2e-553b-4bc6-bc18-3b0626c17a1e,-0.439465,-0.559614,0.502278,0.807046,1.343111,-0.026743
1,d2054499-4dd2-4ea1-9dae-7751c37f58e5,0.120927,-1.074762,0.205903,1.037645,0.220430,0.220944
2,f09ccb39-f576-42af-9803-e5d3c10fdfba,-0.159269,1.191890,0.798654,-1.409790,0.657383,0.681871
3,341a81d8-db2d-4017-95ca-f11d39c62331,-0.439465,-1.383851,-0.683223,-1.035278,0.005974,0.183160
4,68975dd8-703c-43ba-8247-1e6cde4fe067,0.120927,-0.147495,0.798654,0.288590,0.551271,0.357628


Number of entries: 81

SHORT_ANS
Mean Words: 436


,ID,judge,score
0,7a4dfc2e-553b-4bc6-bc18-3b0626c17a1e,Human,-0.439465
1,d2054499-4dd2-4ea1-9dae-7751c37f58e5,Human,0.120927
2,f09ccb39-f576-42af-9803-e5d3c10fdfba,Human,-0.159269
3,341a81d8-db2d-4017-95ca-f11d39c62331,Human,-0.439465
4,68975dd8-703c-43ba-8247-1e6cde4fe067,Human,0.120927


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.469717,2.771568,80,81,0.000004,"[0.28, 0.62]"
ICC2,Single random raters,0.467995,2.738156,80,80,0.000005,"[0.28, 0.62]"
ICC3,Single fixed raters,0.464977,2.738156,80,80,0.000005,"[0.28, 0.62]"
ICC1k,Average raters absolute,0.639193,2.771568,80,81,0.000004,"[0.44, 0.77]"
ICC2k,Average random raters,0.637598,2.738156,80,80,0.000005,"[0.44, 0.77]"
ICC3k,Average fixed raters,0.634791,2.738156,80,80,0.000005,"[0.43, 0.77]"


...

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.194317,1.482365,91,92,0.030563,"[-0.01, 0.38]"
ICC2,Single random raters,0.190936,1.467145,91,91,0.034549,"[-0.02, 0.38]"
ICC3,Single fixed raters,0.189346,1.467145,91,91,0.034549,"[-0.02, 0.38]"
ICC1k,Average raters absolute,0.325402,1.482365,91,92,0.030563,"[-0.02, 0.55]"
ICC2k,Average random raters,0.320648,1.467145,91,91,0.034549,"[-0.03, 0.55]"
ICC3k,Average fixed raters,0.318404,1.467145,91,91,0.034549,"[-0.03, 0.55]"


Cohen's kappa for rounded(z scores x 10): 0.1986696787148594
Algo score, swaps needed: 1729.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2098, 2279, 2321, 2313, 1974, 1858, 2197, 2276, 1821, 2032]
Pseudo-random, avergae swaps needed: 2045.02919

Q2
Mean Words: 2137


,ID,judge,score
0,0568651d-0e40-4bf5-8e24-d8f16403f49e,Human,-0.127540
1,0091731f-bae9-4326-a66a-a744137584a2,Human,-1.477259
2,83cb5f25-30c8-4aff-b6e7-1b9e323e657c,Human,0.065277
3,546ad478-6cd2-4ad5-92c6-2f32ef76d808,Human,-0.705991
4,f9b1bd52-ca9b-4c23-8758-7dcb7006a1b1,Human,-0.705991


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.196336,1.488603,91,92,0.029217,"[-0.01, 0.38]"
ICC2,Single random raters,0.193431,1.475380,91,91,0.032580,"[-0.01, 0.38]"
ICC3,Single fixed raters,0.192043,1.475380,91,91,0.032580,"[-0.01, 0.38]"
ICC1k,Average raters absolute,0.328229,1.488603,91,92,0.029217,"[-0.01, 0.56]"
ICC2k,Average random raters,0.324160,1.475380,91,91,0.032580,"[-0.02, 0.55]"
ICC3k,Average fixed raters,0.322208,1.475380,91,91,0.032580,"[-0.02, 0.55]"


Cohen's kappa for rounded(z scores x 10): 0.2047859977228783
Algo score, swaps needed: 1753.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1804, 1946, 2214, 2029, 2130, 2049, 2043, 1926, 1742, 2155]
Pseudo-random, avergae swaps needed: 1993.26688

Swaps (Machine):
[1004.0, 1176.0, 898.0, 1271.0, 1196.0, 92.0, 919.0, 750.0, 1184.0, 905.0, 831.0, 1729.0, 1753.0]
N: 13 	Mean:  1054.4615384615386 	Var:  167547.01775147932
Avergae swaps (Pseudo-random):
[1499.66689, 1566.66396, 1560.97148, 1837.65831, 1757.52432, 182.57292, 1324.95001, 1091.64396, 1472.83579, 1450.35153, 1457.60248, 2045.02919, 1993.26688]
N: 13 	Mean:  1480.0567476923075 	Var:  204746.44657664374 

1 ) 1499.66689 > 1004.0 => 39.59527459341845
2 ) 1566.66396 > 1176.0 => 28.487920189828877
3 ) 1560.97148 > 898.0 => 53.92266526002976
4 ) 1837.65831 > 1271.0 => 36.45677675009577
5 ) 1757.52432 > 1196.0 => 38.02401870860505
6 ) 182.57292 > 92.0 => 65.9736728589258
7 ) 1324.95001 > 919.0 => 36.18173383461426
8 ) 

In [20]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,e8c7236a-2cae-40ea-8d17-5405ce4a0e0f,12,17,14,90,80,83
1,cfa09378-0303-42fb-9ff1-066c8e2f7e88,4,14,12,70,83,80
2,4fda7a79-36eb-4380-923e-3878eaadddde,7,22,12,70,70,80
3,8b2e671a-8164-4467-9f19-a6dc7cbf313c,4,27,16,90,83,90
4,c6b6baea-3f79-4d14-bc22-83d685e04cfb,9,30,22,83,80,80


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1063.58667
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1416, 1695, 1575, 1607, 1510, 1478, 1474, 1643, 1455, 1365]
Pseudo-random, avergae swaps needed: 1499.33637

Q1
Mean Words: 2048
Algo score, swaps needed: 1364.57761
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1670, 1440, 1581, 1430, 1473, 1577, 1719, 1602, 1378, 1392]
Pseudo-random, avergae swaps needed: 1566.50795

Q2
Mean Words: 947
Algo score, swaps needed: 1112.97992
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1358, 1589, 1523, 1634, 1553, 1586, 1582, 1622, 1535, 1484]
Pseudo-random, avergae swaps needed: 1559.56629

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,1b15cca5-b292-4c29-8e83-384ed76e7b77,20.0,18.0,77,77
1,da455e55-d2ac-4054-aae9-e62723a9dc6c,17.0,17.0,73,83
2,d43eda57-827c-4f18-af5f-dcd5b9660eec,24.0,21.0,80,83
3,9db4494e-5ddd-4a7d-b3e9-031a33003a02,22.0,25.0,90,87
4,346c7a84-2b02-4aaa-81f4-4ae8633b9b69,24.0,25.0,87,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1395.29161
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1914, 1909, 1743, 1821, 1602, 1776, 2085, 1889, 2085, 1876]
Pseudo-random, avergae swaps needed: 1837.29656

Q2
Mean Words: 1546
Algo score, swaps needed: 1554.92878
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1740, 1819, 1511, 1745, 1815, 1743, 1948, 1765, 1956, 1831]
Pseudo-random, avergae swaps needed: 1756.82817

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 959.03298
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1200, 1360, 1517, 1144, 1154, 1326, 1191, 1400, 1227, 1367]
Pseudo-random, avergae swaps needed: 1324.14649

Q2
Mean Words: 570
Algo score, swaps needed: 1046.08702
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1441, 1139, 1282, 1129, 1251, 1229, 1027, 1248, 1286, 1377]
Pseudo-random, avergae swaps needed: 1219.73759

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,39268066-b798-49dd-8159-eda5299389ef,61,16,24,77,80,77
1,cee3d697-38c2-403f-a5e8-2bf351cced67,60,11,18,87,87,83
2,5a6f914f-609c-4043-b2ba-6ad26b7fb36c,41,4,25,87,87,83
3,ab397bfe-7c68-4f02-baee-46407830b974,36,28,22,83,83,83
4,ee5b756f-84b7-4707-ab9a-a9064b65ebe2,43,25,16,77,77,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1073.8407
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1327, 1577, 1514, 1633, 1585, 1558, 1714, 1623, 1338, 1434]
Pseudo-random, avergae swaps needed: 1472.20717

Q2
Mean Words: 1050
Algo score, swaps needed: 1225.50466
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1508, 1353, 1572, 1447, 1486, 1367, 1369, 1511, 1357, 1354]
Pseudo-random, avergae swaps needed: 1450.233

Q3
Mean Words: 839
Algo score, swaps needed: 1030.00018
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1336, 1551, 1475, 1520, 1395, 1334, 1650, 1456, 1199, 1303]
Pseudo-random, avergae swaps needed: 1457.32272

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,1f8baf28-0b57-47fe-bddb-0a27b92be278,14.5,19.0,73,80
1,3608d390-675a-4e40-afc1-1f17fdebe8e8,23.5,22.0,83,90
2,fcfd3df1-72e1-4e8a-b798-68fb0162b8b8,18.5,21.0,87,87
3,90104063-f02a-4ab0-b693-12b727a108e5,21.0,15.0,70,73
4,a577fd48-7230-4784-a7eb-6e268b042b9c,13.0,16.0,80,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1977.50891
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2138, 1921, 2000, 2165, 2030, 2339, 2295, 2168, 2084, 1891]
Pseudo-random, avergae swaps needed: 2045.7049

Q2
Mean Words: 2137
Algo score, swaps needed: 1852.88044
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1987, 2261, 1935, 1799, 1967, 2180, 1988, 1884, 1907, 2168]
Pseudo-random, avergae swaps needed: 1993.2798

Swaps (Machine):
[1063.58667, 1364.57761, 1112.97992, 1395.29161, 1554.92878, 107.99537, 959.03298, 1046.08702, 1073.8407, 1225.50466, 1030.00018, 1977.50891, 1852.88044]
N: 13 	Mean:  1212.6319115384617 	Var:  197367.21358401564
Avergae swaps (Pseudo-random):
[1499.33637, 1566.50795, 1559.56629, 1837.29656, 1756.82817, 182.46159, 1324.14649, 1219.73759, 1472.20717, 1450.233, 1457.32272, 2045.7049, 1993.2798]
N: 13 	Mean:  1489.5868153846154 	Var:  198296.9387529382 

1 ) 1499.33637 > 1063.58667 => 34.00411898439214
2 )

In [21]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,cfa09378-0303-42fb-9ff1-066c8e2f7e88,4,14,12,77,87,87
1,b4b58e7e-2a41-4e45-aca3-7b69511f3866,12,39,24,87,67,80
2,45942e14-a556-414d-b4d7-9287409db104,13,43,19,80,83,83
3,dcef7f09-f784-48cb-b65e-59d0293f4195,10,30,32,87,83,87
4,c74cd4a8-869a-4714-892e-be61183e7835,4,27,24,90,87,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1327.50167
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1528, 1180, 1568, 1544, 1479, 1658, 1465, 1672, 1642, 1495]
Pseudo-random, avergae swaps needed: 1499.65638

Q1
Mean Words: 2048
Algo score, swaps needed: 1283.97738
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1542, 1534, 1714, 1654, 1706, 1472, 1491, 1555, 1663, 1650]
Pseudo-random, avergae swaps needed: 1566.25132

Q2
Mean Words: 947
Algo score, swaps needed: 1217.39444
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1452, 1579, 1374, 1611, 1422, 1657, 1536, 1609, 1729, 1509]
Pseudo-random, avergae swaps needed: 1559.69437

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,ecdebb90-4db9-4967-b6f9-ea3759679fcc,13.0,16.0,83,80
1,0cfd3781-8b12-4e17-8d9d-16c15ce0f246,12.5,18.0,87,87
2,6726ea7e-8d3e-46f5-bb70-5aff7c5b6402,9.0,15.0,80,87
3,a9738fe5-2df1-4661-b147-de435b7eeeb6,17.0,20.0,77,87
4,79fc7532-8d8e-4995-adbc-23e3aa4c75af,11.5,11.0,83,77


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1578.45767
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2188, 1949, 1709, 1739, 1654, 1537, 1849, 1894, 1901, 1999]
Pseudo-random, avergae swaps needed: 1838.07482

Q2
Mean Words: 1546
Algo score, swaps needed: 1572.3984
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1861, 1911, 1747, 1763, 1721, 2084, 1601, 1552, 1730, 1722]
Pseudo-random, avergae swaps needed: 1757.55939

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 926.46827
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1257, 1476, 1332, 1302, 1213, 1242, 1292, 1587, 1375, 1339]
Pseudo-random, avergae swaps needed: 1324.23719

Q2
Mean Words: 570
Algo score, swaps needed: 1010.98591
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1294, 1186, 1123, 1187, 1364, 1105, 1404, 1091, 1111, 1172]
Pseudo-random, avergae swaps needed: 1219.24378

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,bb065bd5-206a-4c0f-ba0f-f193651f5ac6,62,23,40,77,73,90
1,7fc61220-6c30-428e-9cbe-ba29c1d2e8fb,62,40,32,90,83,80
2,aa684706-1bbf-47a8-8828-62f8fde0f3c7,53,27,23,77,70,80
3,6c715ed9-9520-4c0d-8d4f-b56ceef0320a,53,43,15,83,83,80
4,4556f34c-b15d-4f3e-b818-09cff2ec3400,54,28,6,67,77,63


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1355.55177
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1262, 1592, 1482, 1730, 1432, 1467, 1540, 1663, 1254, 1458]
Pseudo-random, avergae swaps needed: 1472.71635

Q2
Mean Words: 1050
Algo score, swaps needed: 1163.31542
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1685, 1454, 1395, 1511, 1317, 1490, 1587, 1411, 1326, 1382]
Pseudo-random, avergae swaps needed: 1449.21663

Q3
Mean Words: 839
Algo score, swaps needed: 1059.66915
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1593, 1364, 1310, 1344, 1345, 1578, 1490, 1285, 1628, 1401]
Pseudo-random, avergae swaps needed: 1458.5898

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,509aec91-34c1-4d28-b94d-f7845bfe5226,14.0,19.0,67,70
1,a577fd48-7230-4784-a7eb-6e268b042b9c,13.0,16.0,80,80
2,1f8baf28-0b57-47fe-bddb-0a27b92be278,14.5,19.0,63,80
3,83c20b41-645b-4931-97bf-870d498e43de,14.0,8.0,87,80
4,ba2dd55c-5abd-4f7f-8560-18efd2679188,23.0,27.0,80,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1718.97797
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1936, 2171, 2226, 2158, 1791, 1995, 2136, 2242, 2061, 2156]
Pseudo-random, avergae swaps needed: 2044.98433

Q2
Mean Words: 2137
Algo score, swaps needed: 1802.09925
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2217, 2013, 1930, 2002, 2257, 2055, 1930, 1918, 1862, 1917]
Pseudo-random, avergae swaps needed: 1993.58644

Swaps (Machine):
[1327.50167, 1283.97738, 1217.39444, 1578.45767, 1572.3984, 108.49357, 926.46827, 1010.98591, 1355.55177, 1163.31542, 1059.66915, 1718.97797, 1802.09925]
N: 13 	Mean:  1240.40699 	Var:  174101.9627388273
Avergae swaps (Pseudo-random):
[1499.65638, 1566.25132, 1559.69437, 1838.07482, 1757.55939, 182.49323, 1324.23719, 1219.24378, 1472.71635, 1449.21663, 1458.5898, 2044.98433, 1993.58644]
N: 13 	Mean:  1489.7156946153846 	Var:  198340.275571395 

1 ) 1499.65638 > 1327.50167 => 12.178640667082606
2 ) 156

In [22]:
score_exams(exams,model='nlp3',normv=1,score=2,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']
Leaving z scores in place
Leaving z scores in place
Leaving z scores in place
Computing z scores for human scoring


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,afaca025-5d82-46e8-bfba-a1ce2167adca,-0.159269,0.470683,0.502278,0.144799,1.420513,-0.126103
1,d2054499-4dd2-4ea1-9dae-7751c37f58e5,0.120927,-1.074762,0.205903,0.994781,-0.083617,0.661264
2,5b58226f-9ae5-4ad2-8261-d659f95da503,0.681318,0.161594,0.354091,-0.413266,-1.325318,-0.985834
3,e021f764-a5d2-432a-89f5-00ba2b760f0f,0.681318,0.779772,1.243217,1.063094,0.701413,0.702417
4,07ac7940-e702-498a-a97c-333cb233a847,-1.280052,-0.971732,-1.572349,-2.286492,-3.295831,-1.693152


Number of entries: 81

SHORT_ANS
Mean Words: 436


,ID,judge,score
0,afaca025-5d82-46e8-bfba-a1ce2167adca,Human,-0.159269
1,065476cf-e25b-4039-a5b0-69bfd27e61dd,Human,-0.999856
2,d2054499-4dd2-4ea1-9dae-7751c37f58e5,Human,0.120927
3,5b58226f-9ae5-4ad2-8261-d659f95da503,Human,0.681318
4,e021f764-a5d2-432a-89f5-00ba2b760f0f,Human,0.681318


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.260254,1.703632,80,81,0.008882,"[0.05, 0.45]"
ICC2,Single random raters,0.256877,1.682955,80,80,0.010504,"[0.04, 0.45]"
ICC3,Single fixed raters,0.254553,1.682955,80,80,0.010504,"[0.04, 0.45]"
ICC1k,Average raters absolute,0.413019,1.703632,80,81,0.008882,"[0.09, 0.62]"
ICC2k,Average random raters,0.408755,1.682955,80,80,0.010504,"[0.08, 0.62]"
ICC3k,Average fixed raters,0.405807,1.682955,80,80,0.010504,"[0.08, 0.62]"


...

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.182515,1.446528,91,92,0.039475,"[-0.02, 0.37]"
ICC2,Single random raters,0.179031,1.431663,91,91,0.044348,"[-0.03, 0.37]"
ICC3,Single fixed raters,0.177518,1.431663,91,91,0.044348,"[-0.03, 0.37]"
ICC1k,Average raters absolute,0.308689,1.446528,91,92,0.039475,"[-0.04, 0.54]"
ICC2k,Average random raters,0.303692,1.431663,91,91,0.044348,"[-0.06, 0.54]"
ICC3k,Average fixed raters,0.301512,1.431663,91,91,0.044348,"[-0.06, 0.54]"


Cohen's kappa for rounded(z scores x 10): 0.17165245801214968
Algo score, swaps needed: 1755.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2298, 2055, 2016, 2089, 2077, 2159, 1799, 1980, 1914, 1988]
Pseudo-random, avergae swaps needed: 2045.44676

Q2
Mean Words: 2137


,ID,judge,score
0,e1308431-d29b-47a7-8201-12fd04f5cac9,Human,0.258094
1,7b74720b-c2e8-4eb3-9274-ea3fb17441e2,Human,-0.127540
2,348bfa7e-1c58-4b5b-9b11-af9b507ee3cb,Human,-0.127540
3,e63e6f2b-24aa-45e2-8cf4-ceba745a1010,Human,-0.898808
4,f0166970-fdaf-4f6b-9ae6-e8e25ef8b198,Human,-0.513174


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.217202,1.554939,91,92,0.01794,"[0.01, 0.4]"
ICC2,Single random raters,0.214463,1.541209,91,91,0.02020,"[0.01, 0.4]"
ICC3,Single fixed raters,0.212973,1.541209,91,91,0.02020,"[0.01, 0.4]"
ICC1k,Average raters absolute,0.356888,1.554939,91,92,0.01794,"[0.03, 0.57]"
ICC2k,Average random raters,0.353182,1.541209,91,91,0.02020,"[0.02, 0.57]"
ICC3k,Average fixed raters,0.351159,1.541209,91,91,0.02020,"[0.02, 0.57]"


Cohen's kappa for rounded(z scores x 10): 0.16671429748632016
Algo score, swaps needed: 1796.0
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1891, 1725, 2297, 1936, 1873, 2096, 2122, 2058, 2143, 1717]
Pseudo-random, avergae swaps needed: 1994.1412

Swaps (Machine):
[1261.0, 1261.0, 1165.0, 1571.0, 1500.0, 101.0, 891.0, 927.0, 1337.0, 1112.0, 1040.0, 1755.0, 1796.0]
N: 13 	Mean:  1209.0 	Var:  179109.23076923078
Avergae swaps (Pseudo-random):
[1499.90796, 1566.48321, 1560.01438, 1837.17309, 1756.61942, 182.54523, 1324.22571, 1091.79753, 1472.17265, 1449.68919, 1457.75468, 2045.44676, 1994.1412]
N: 13 	Mean:  1479.843923846154 	Var:  204790.024822612 

1 ) 1499.90796 > 1261.0 => 17.306477684971433
2 ) 1566.48321 > 1261.0 => 21.608136092167992
3 ) 1560.01438 > 1165.0 => 28.991728109706347
4 ) 1837.17309 > 1571.0 => 15.619693188763485
5 ) 1756.61942 > 1500.0 => 15.759865486523445
6 ) 182.54523 > 101.0 => 57.51832256180081
7 ) 1324.22571 > 891.0 => 39.11345991014162
8 ) 1091.

In [23]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,ID,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,c74cd4a8-869a-4714-892e-be61183e7835,4,27,24,87,87,80
1,88b8c7fa-b807-4ab3-b690-602e4b0787aa,6,18,20,83,87,83
2,afaca025-5d82-46e8-bfba-a1ce2167adca,6,28,18,90,87,83
3,de219fed-6a0e-4796-875c-300be31e1347,11,23,8,73,80,77
4,01759358-64dd-4801-b923-5a9914b62f19,6,33,19,87,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1362.98474
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1516, 1345, 1513, 1286, 1490, 1488, 1463, 1562, 1705, 1500]
Pseudo-random, avergae swaps needed: 1499.459

Q1
Mean Words: 2048
Algo score, swaps needed: 1259.61929
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1469, 1559, 1586, 1635, 1725, 1580, 1492, 1626, 1477, 1518]
Pseudo-random, avergae swaps needed: 1566.4181

Q2
Mean Words: 947
Algo score, swaps needed: 1312.33292
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1542, 1433, 1483, 1537, 1357, 1666, 1590, 1522, 1510, 1596]
Pseudo-random, avergae swaps needed: 1560.06736

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,4ef2af3a-05d9-4992-9746-a24945431d7b,16.0,23.0,77,60
1,0b4a8297-98e1-4153-bddb-d6562cd5041e,13.5,22.0,80,77
2,5b2a6aa4-fdf6-40fc-880c-09a15c2fe77b,12.0,21.0,83,80
3,732a8471-dca7-4b81-be0c-606cc6e86d0a,22.0,16.0,77,83
4,6dc8ed5f-bc91-4ed5-bc1e-2de723a3aae9,11.0,13.0,83,83


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1426.94488
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1883, 1719, 1765, 1931, 2222, 1610, 2100, 1847, 2133, 1695]
Pseudo-random, avergae swaps needed: 1837.57631

Q2
Mean Words: 1546
Algo score, swaps needed: 1692.05737
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1819, 1696, 2002, 2010, 1823, 1655, 1400, 1700, 1796, 1394]
Pseudo-random, avergae swaps needed: 1758.10921

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 1075.06208
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1296, 1434, 1373, 1328, 1232, 1426, 1332, 1390, 1110, 1508]
Pseudo-random, avergae swaps needed: 1324.45835

Q2
Mean Words: 570
Algo score, swaps needed: 1107.89282
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1227, 1052, 1203, 1293, 1183, 1170, 1071, 1094, 1185, 1243]
Pseudo-random, avergae swaps needed: 1219.53493

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,ID,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,1b273341-a255-40ef-9a90-17fbdf05af4b,48,39,31,83,83,87
1,ed6cf722-94b9-4474-8bf7-318cb15713b8,67,25,28,70,83,87
2,4c68352f-27a5-49c2-aecd-887e788ab441,55,15,23,77,77,83
3,6266e45d-2172-43e6-98f5-f9ed6d82b414,62,31,29,87,90,87
4,c04e80c3-8a55-408c-afd7-c6d5197b5fd2,50,40,18,83,77,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1334.62252
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1595, 1470, 1469, 1542, 1520, 1501, 1542, 1452, 1287, 1395]
Pseudo-random, avergae swaps needed: 1471.70444

Q2
Mean Words: 1050
Algo score, swaps needed: 1083.63437
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1337, 1567, 1517, 1591, 1493, 1389, 1308, 1467, 1237, 1405]
Pseudo-random, avergae swaps needed: 1449.61455

Q3
Mean Words: 839
Algo score, swaps needed: 1002.36012
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1443, 1478, 1428, 1579, 1330, 1318, 1407, 1477, 1443, 1369]
Pseudo-random, avergae swaps needed: 1458.0963

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,ID,Q1_x,Q2_x,Q1_y,Q2_y
0,19504cf2-0e8e-47e7-8cd8-37ccc42cc860,20.5,15.0,73,80
1,e1308431-d29b-47a7-8201-12fd04f5cac9,22.0,17.0,77,83
2,9db89209-0865-45ae-a102-45967994336f,21.0,14.0,90,83
3,85ca12a0-f4e5-4045-8350-676e0e159221,22.0,17.0,97,73
4,0a1cbcb8-fb01-45be-bb70-9c97770f647c,18.0,11.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1803.05968
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1878, 2178, 1916, 2063, 1978, 2051, 2153, 2108, 2209, 2108]
Pseudo-random, avergae swaps needed: 2045.09657

Q2
Mean Words: 2137
Algo score, swaps needed: 1807.38171
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2117, 2052, 1817, 2172, 2064, 2062, 2176, 1942, 2130, 2155]
Pseudo-random, avergae swaps needed: 1993.39235

Swaps (Machine):
[1362.98474, 1259.61929, 1312.33292, 1426.94488, 1692.05737, 134.47636, 1075.06208, 1107.89282, 1334.62252, 1083.63437, 1002.36012, 1803.05968, 1807.38171]
N: 13 	Mean:  1261.725296923077 	Var:  173044.9624971154
Avergae swaps (Pseudo-random):
[1499.459, 1566.4181, 1560.06736, 1837.57631, 1758.10921, 182.52418, 1324.45835, 1219.53493, 1471.70444, 1449.61455, 1458.0963, 2045.09657, 1993.39235]
N: 13 	Mean:  1489.696280769231 	Var:  198315.17725239138 

1 ) 1499.459 > 1362.98474 => 9.535506888250664
2 )

[back to contents](#Contents)